In [1]:
import pandas as pd
import numpy as np
from flowenv.src.const import Const
from pathlib import Path
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTENC


def min_max_p(p):
    min_p = p.min()
    max_p = p.max()
    return (p - min_p) / (max_p - min_p)


CONST = Const()
ATTACK_LABELS = CONST.attack_labels
CATEGORICAL_FEATURES = ["Dst Port", "Protocol"]

TRAIN_DATA_PATH = "./DNP3_Intrusion_Detection_Dataset_Final/Training_Testing_Balanced_CSV_Files/CICFlowMeter/CICFlowMeter_Training_Balanced.csv"
TEST_DATA_PATH = "./DNP3_Intrusion_Detection_Dataset_Final/Training_Testing_Balanced_CSV_Files/CICFlowMeter/CICFlowMeter_Testing_Balanced.csv"

# TRAIN_DATA_PATH = Path(__file__).resolve().parent.joinpath(TRAIN_DATA_PATH)
# TEST_DATA_PATH = Path(__file__).resolve().parent.joinpath(TEST_DATA_PATH)

train_data = pd.read_csv(TRAIN_DATA_PATH).replace([np.inf, -np.inf], np.nan).dropna(how="all").dropna(how="all", axis=1)
test_data = pd.read_csv(TEST_DATA_PATH).dropna(how="all").replace([np.inf, -np.inf], np.nan).dropna(how="all", axis=1)


In [4]:
train_data.duplicated().sum()

np.int64(340)

In [5]:
binarize = False
balance = True

train_data = pd.read_csv(TRAIN_DATA_PATH).replace([np.inf, -np.inf], np.nan).dropna(how="all").dropna(how="all", axis=1)
test_data = pd.read_csv(TEST_DATA_PATH).replace([np.inf, -np.inf], np.nan).dropna(how="all").dropna(how="all", axis=1)

# unique
targets = ""
if binarize:
    train_data["Binary Label"] = train_data["Label"].apply(lambda label: label in ATTACK_LABELS)
    test_data["Binary Label"] = test_data["Label"].apply(lambda label: label in ATTACK_LABELS)
    targets = "Binary Label"
else:
    train_data["Label Index"] = train_data["Label"].apply(lambda x: 0 if x not in ATTACK_LABELS else ATTACK_LABELS.index(x) + 1)
    test_data["Label Index"] = test_data["Label"].apply(lambda x: 0 if x not in ATTACK_LABELS else ATTACK_LABELS.index(x) + 1)
    targets = "Label Index"

# --------

train_data = train_data.filter(items=CONST.features_labels + [targets])
test_data = test_data.filter(items=CONST.features_labels + [targets])


In [13]:
conbine_data = pd.concat([train_data, test_data], ignore_index=True)

In [14]:
ohe = OneHotEncoder(sparse_output=False)
conbine_data_ohe = ohe.fit_transform(conbine_data[CATEGORICAL_FEATURES])
conbine_data_ohe = pd.DataFrame(conbine_data_ohe, columns=ohe.get_feature_names_out(CATEGORICAL_FEATURES))

In [15]:
drop_columns_data = conbine_data.drop(columns=CATEGORICAL_FEATURES)
conbine_data_after_ohe = pd.concat([drop_columns_data, conbine_data_ohe], axis=1)

In [ ]:

ohe = OneHotEncoder(sparse_output=False)
train_data_ohe = ohe.fit_transform(train_data[CATEGORICAL_FEATURES])
train_data_ohe = pd.DataFrame(train_data_ohe, columns=ohe.get_feature_names_out(CATEGORICAL_FEATURES))
train_data = pd.concat([train_data.drop(columns=CATEGORICAL_FEATURES), train_data_ohe], axis=1)
train_smotenc_columns = ohe.get_feature_names_out(CATEGORICAL_FEATURES).tolist()

test_data_ohe = ohe.fit_transform(test_data[CATEGORICAL_FEATURES])
test_data_ohe = pd.DataFrame(test_data_ohe, columns=ohe.get_feature_names_out(CATEGORICAL_FEATURES))
test_data = pd.concat([test_data.drop(columns=CATEGORICAL_FEATURES), test_data_ohe], axis=1)

for label in CONST.normalization_features:
    train_data[label] = min_max_p(train_data[label])
    test_data[label] = min_max_p(test_data[label])

train_data = train_data.dropna(how="any")
test_data = test_data.dropna(how="any")


In [3]:
y_train = train_data[targets]
X_train = train_data.drop(columns=[targets])

smote = SMOTENC(
    categorical_features=[X_train.columns.get_loc(label) for label in train_smotenc_columns], 
    # random_state=42,
    # k_neighbors=3
)

In [33]:
print(X_train.shape)
print(y_train)
print([train_data.columns.get_loc(label) for label in train_smotenc_columns])

(5126, 493)
0       0
1       0
2       0
3       2
4       1
       ..
5121    2
5122    1
5123    2
5124    2
5125    0
Name: Label Index, Length: 5126, dtype: int64
[69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 24

In [4]:
X_train, y_train = smote.fit_resample(X_train, y_train)

X_resampled = pd.DataFrame(X_train, columns=CONST.features_labels)
y_resampled = pd.Series(y_train, name=targets)

resampled_train_data = pd.concat([X_resampled, y_resampled], axis=1)